# DEA as a sign-constrained variant of the CNLS problem

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : June 17, 2020

References:

[1] Kuosmanen, T. and Johnson, A. L. (2010). Data envelopment analysis as nonparametric least-squares regression. Operations Research, 58, pp. 149–160.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import DEA2CNLS

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(10)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03
5,1483,1587,3395,295,953,22949,0.65
6,658,570,1333,44,917,3599,0.11
7,1433,1311,3518,171,1580,11081,0.16
8,850,564,1415,98,116,377,1.00
9,1155,1108,2469,203,740,10134,0.64


In [4]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [5]:
# define and solve the CNLS model

model = DEA2CNLS.dea2cnls(y, x)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 7921            
  Cones                  : 0               
  Scalar variables       : 356             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 35
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0              

In [6]:
# retrive the residuals (additive DEA efficiency measure)
val = list(model.e[:].value)
eps = np.asarray(val)

In [10]:
# multiplicative DEA efficiency measure (c.f., DEA_oo_vrs)
theta = 1 - eps/y

**Note**: In the single-output setting, the additive DEA efficiency measure (i.e., this case) is equivalent to the multiplicative DEA efficiency measure (i.e., [Radial output oriented VRS model](https://nbviewer.jupyter.org/github/ds2010/pyStoNED-Tutorials/blob/master/DEA/DEA_oo_vrs.ipynb)) in the sense that:
$\theta_i^{DEA} = 1 - eps_i^{DEA}/y_i$

In [9]:
theta

0     2.072203
1     2.081837
2     2.428727
3     2.161610
4     3.680143
        ...   
84    1.929890
85    2.534116
86    2.484602
87    4.680241
88    1.897683
Name: Energy, Length: 89, dtype: float64